In [1]:
import os, json
import gc

import pandas as pd
import numpy as np
from scipy.sparse import coo_array
import duckdb
from tqdm.notebook import tqdm


In [2]:
for i in range(10):
    cwd = os.getcwd()
    r, c = os.path.split(cwd)
    if c == 'repo':
        print(f'new working dir: {cwd}')
        break
    os.chdir(r)

new working dir: c:\Users\aknof\Documents\GT\CSE_6250_BD4H\Project\repo


In [3]:
con = duckdb.connect()

CUI_MIN_OCC_POP = 1  #100
CUI_MAX_OCC_PAT = 10000
BILL_MIN_OCC_POP = 1  #1000

In [4]:
file_dir = os.path.join('data', 'i2b2_metamap_out')
all_pat_df = None
for i, file in enumerate(os.listdir(file_dir)):
    with open(os.path.join(file_dir, file), 'r') as infile:
        patient_cuis = json.load(infile)
    break 
patient_cuis.keys()

dict_keys(['AllDocuments', 'metadata'])

In [9]:
patient_cuis['AllDocuments'][0].keys()

dict_keys(['Document'])

In [10]:
patient_cuis['AllDocuments'][0]['Document'].keys()

dict_keys(['CmdLine', 'AAs', 'Negations', 'Utterances'])

In [12]:
patient_cuis['AllDocuments'][0]['Document']['Utterances'][0].keys()

dict_keys(['PMID', 'UttSection', 'UttNum', 'UttText', 'UttStartPos', 'UttLength', 'Phrases'])

In [14]:
patient_cuis['AllDocuments'][0]['Document']['Utterances'][0]['Phrases'][0].keys()

dict_keys(['PhraseText', 'SyntaxUnits', 'PhraseStartPos', 'PhraseLength', 'Candidates', 'Mappings'])

In [33]:
for p in patient_cuis['AllDocuments'][0]['Document']['Utterances'][0]['Phrases']:
    if p['Candidates'] or p['Mappings']:
        for m in p['Mappings'][0]['MappingCandidates']:
            print(m)
            # for k, v in m.items():
            #     print(k, v)
        # print(p)
            break

{'CandidateScore': '-840', 'CandidateCUI': 'C1302393', 'CandidateMatched': 'Admission date', 'CandidatePreferred': 'Date of admission', 'MatchedWords': ['admission', 'date'], 'SemTypes': ['tmco'], 'MatchMaps': [{'TextMatchStart': '4', 'TextMatchEnd': '4', 'ConcMatchStart': '1', 'ConcMatchEnd': '1', 'LexVariation': '0'}, {'TextMatchStart': '5', 'TextMatchEnd': '5', 'ConcMatchStart': '2', 'ConcMatchEnd': '2', 'LexVariation': '0'}], 'IsHead': 'yes', 'IsOverMatch': 'no', 'Sources': ['LNC', 'MTH', 'SNOMEDCT_US'], 'ConceptPIs': [{'StartPos': '93', 'Length': '14'}], 'Status': '0', 'Negated': '0'}
{'CandidateScore': '-840', 'CandidateCUI': 'C0586177', 'CandidateMatched': 'Report status', 'CandidatePreferred': 'Report status (finding)', 'MatchedWords': ['report', 'status'], 'SemTypes': ['fndg'], 'MatchMaps': [{'TextMatchStart': '4', 'TextMatchEnd': '4', 'ConcMatchStart': '1', 'ConcMatchEnd': '1', 'LexVariation': '0'}, {'TextMatchStart': '5', 'TextMatchEnd': '5', 'ConcMatchStart': '2', 'ConcMatc

In [155]:
del patient_cuis
gc.collect()

3110

In [4]:
def get_first_candidate(cui_dicts):
    cui = None
    for candidate in cui_dicts:
        if 'CandidateCUI' in candidate:
            cui = candidate['CandidateCUI']
            if cui:
                break
    return cui

In [5]:
def get_patient_cuis(patient_json):
    cuis = []
    for doc in patient_json['AllDocuments']:
        for utterance in doc['Document']['Utterances']:
            for phrase in utterance['Phrases']:
                for map in phrase['Mappings']:
                    cui = get_first_candidate(map['MappingCandidates'])
                    if cui:
                        cuis.append(cui)
    patient_df = pd.DataFrame({'cuis': np.unique(np.array(cuis))})
    return patient_df

In [7]:
err_cnt, empty_cnt = 0, 0
errors = []
file_dir = os.path.join('data', 'i2b2_metamap_out')
all_pat_raw_df = None
for i, file in tqdm(enumerate(os.listdir(file_dir))):
    meta_pairs = file.split('.')[0].split('_')
    with open(os.path.join(file_dir, file), 'r') as infile:
        try:
            patient_json = json.load(infile)
        except json.JSONDecodeError as e:
            patient_txt = infile.read()
            if len(patient_txt):
                err_cnt += 1
                errors.append({'file': file, 'error': e})
            else:
                empty_cnt +=1
                continue
    patient_df = get_patient_cuis(patient_json)
    for p in meta_pairs:
        if p:
            k, v = p.split('-')
            patient_df[f'{k}'] = v
    if all_pat_raw_df is None:
        all_pat_raw_df = patient_df
    else:
        all_pat_raw_df = pd.concat([all_pat_raw_df, patient_df])
err_cnt, empty_cnt

0it [00:00, ?it/s]

(0, 2)

In [8]:
all_pat_raw_df.head()

,cuis,index,row,sub,chunk
0,C0001554,0,0,3,0
1,C0001962,0,0,3,0
2,C0002111,0,0,3,0
3,C0002871,0,0,3,0
4,C0003015,0,0,3,0


In [145]:
all_pat_raw_df.count()

cuis     239336
index    239336
row      239336
sub      239336
chunk    239336
dtype: int64

In [4]:
raw_pat_cui_file = os.path.join('data', 'consumed', 'i2b2_patient_cuis.csv')

In [146]:
all_pat_raw_df.to_csv(raw_pat_cui_file, index_label='df_id')

In [7]:
all_pat_raw_df = pd.read_csv(raw_pat_cui_file).drop(columns=['df_id'])
all_pat_raw_df.head()

,cuis,index,row,sub,chunk
0,C0001554,0,0,3,0
1,C0001962,0,0,3,0
2,C0002111,0,0,3,0
3,C0002871,0,0,3,0
4,C0003015,0,0,3,0


In [8]:
all_pat_df = con.execute(""" 
select cuis as umls
, sub as SUBJECT_ID
, dense_rank() over(order by cast(sub as int)) - 1 as patient_idx
from all_pat_raw_df
group by cuis, sub
""").df()
all_pat_df.count()

umls           222148
SUBJECT_ID     222148
patient_idx    222148
dtype: int64

In [9]:
del all_pat_raw_df
gc.collect()

472

In [148]:
all_pat_df.head()

,umls,SUBJECT_ID,patient_idx
0,C0041260,1,0
1,C0184666,1,0
2,C0439086,1,0
3,C1444662,1,0
4,C2985545,1,0


In [19]:
# file_dir = os.path.join('data', 'nlp_jsl')
# all_pat_df = None
# for file in os.listdir(file_dir):
#     patient = file.split('-')[1]
#     patient_df = pd.read_csv(os.path.join(file_dir, file))
#     patient_df['SUBJECT_ID'] = patient
#     if all_pat_df is None:
#         all_pat_df = patient_df
#     else:
#         all_pat_df = pd.concat([all_pat_df, patient_df])
#     # break
# all_pat_df.head()

,Unnamed: 0.1,Unnamed: 0,sentence,begin,end,chunks,entity,confidence,model,umls,SUBJECT_ID
0,0,0,36,2624,2653,part positional/lead placement,DRUG,0.702100,ner_ade_clinical,['C1140614'],4367
1,1,1,52,3791,3804,lead placement,DRUG,0.779950,ner_ade_clinical,['C1283151'],4367
2,2,2,56,4114,4127,lead placement,DRUG,0.836000,ner_ade_clinical,['C1283151'],4367
3,3,3,91,7368,7383,C/O posterior/lt,DRUG,0.627100,ner_ade_clinical,['C0856256'],4367
4,4,4,91,7403,7421,same w atc tylenol,DRUG,0.615425,ner_ade_clinical,['C3897300'],4367


In [26]:
# all_pat_df['umls'] = all_pat_df['umls'].apply(lambda x: x.split("'")[1])
# all_pat_df = all_pat_df[['umls', 'SUBJECT_ID']]
# all_pat_df.head()

,umls,SUBJECT_ID
0,C1140614,4367
1,C1283151,4367
2,C1283151,4367
3,C0856256,4367
4,C3897300,4367


In [10]:
pat_only_df = con.execute(""" 
select distinct subject_id, patient_idx
from all_pat_df
""").df()
pat_only_df.head()

,SUBJECT_ID,patient_idx
0,1154,1002
1,1155,1003
2,1156,1004
3,1157,1005
4,1158,1006


In [11]:
patient_df = con.execute(f"""
with patient as (
    select * from all_pat_df --pat_unique_df
)
,high_vol as (
    select subject_id from all_pat_df
    group by subject_id
    having count(distinct umls) > {CUI_MAX_OCC_PAT}
)
select distinct umls, subject_id, patient_idx
from patient p
where not exists (select 1 from high_vol h where p.subject_id = h.subject_id)
""").df()
patient_df.head()

,umls,SUBJECT_ID,patient_idx
0,C0027552,1154,1002
1,C4036205,1154,1002
2,C1550028,1154,1002
3,C5394385,1155,1003
4,C0332287,1155,1003


In [12]:
del all_pat_df 
gc.collect()

0

In [1]:
vocab_file = os.path.join('data', 'clinical_vocab', 'MRCONSO.RRF')
cols = ['CUI', 'LAT', 'TS', 'LUI', 'STT', 'SUI', 'ISPREF', 'AUI', 'SAUI',
    'SCUI', 'SDUI', 'SAB', 'TTY', 'CODE', 'STR', 'SRL', 'SUPPRESS', 'CVF']
umls_df = pd.read_csv(vocab_file, sep="|", 
    header=None, names=cols, index_col=False)

# umls_df.head()

In [15]:
umls = con.execute(f"""
select CUI, row_number() over(order by CUI) - 1 as feature_idx
from umls_df as u 
where LAT = 'ENG'
group by CUI
having count(*) >= {CUI_MIN_OCC_POP}
""").df()
umls.head()

,CUI,feature_idx
0,C0000005,0
1,C0000039,1
2,C0000052,2
3,C0000074,3
4,C0000084,4


In [16]:
del umls_df
gc.collect()

0

In [ ]:
umls.to_csv(os.path.join('data', 'consumed', 'feature_map.csv'))

In [14]:
umls = pd.read_csv(os.path.join('data', 'consumed', 'feature_map.csv'))
umls.head()

,Unnamed: 0,CUI,feature_idx
0,0,C0000379,1
1,1,C0000598,2
2,2,C0000726,3
3,3,C0000727,4
4,4,C0000729,5


In [118]:
all_pat_df.count()

umls          222148
SUBJECT_ID    222148
dtype: int64

In [15]:
sparse_df = con.execute(""" 
select p.patient_idx
,u.feature_idx
,count(p.umls) as feat_cnt
from patient_df p
join umls u
on p.umls = u.cui
group by p.patient_idx
,u.feature_idx
""").df()
sparse_df.head()

,patient_idx,feature_idx,feat_cnt
0,78,669,1
1,69,1034,1
2,74,11615,1
3,85,11938,1
4,44,1891,1


In [26]:
sparse_df.to_csv(os.path.join('data', 'consumed', 'i2b2_sparse_feature.csv'), index_label='index')

In [16]:
sparse_shape = con.execute(""" 
select max(patient_idx) + 1 as rows
,max(feature_idx) + 1 as cols
from sparse_df 
""").df()
sparse_shape.head()


,rows,cols
0,1088,15453


In [20]:
sparse_array = coo_array((sparse_df['feat_cnt'].values, 
                        (sparse_df['patient_idx'].values, sparse_df['feature_idx'].values)), 
                        shape=(sparse_shape.at[0, 'rows'], sparse_shape.at[0, 'cols']))

In [32]:
sparse_array_np = sparse_array.toarray()
type(sparse_array_np)

numpy.ndarray

In [28]:
np.save(os.path.join('data', 'consumed', 'i2b2_sparse_feature.npy'), sparse_array_np, allow_pickle=True)

In [28]:
# feature_df = pd.DataFrame(sparse_array)
# feature_df.head()

,0,1,2,3,4,5,6,7,8,9,...,4367337,4367338,4367339,4367340,4367341,4367342,4367343,4367344,4367345,4367346
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
# feature_df.to_csv(os.path.join('data', 'consumed', 'i2b2_sparse_feature.csv'), index_label='patient_idx')

In [122]:
# pat_base_df = con.execute(""" 
# with patients as (
#     select distinct subject_id from all_pat_df limit 100
# )
#     select p.subject_id
#     ,u.feature_idx
#     ,u.CUI
#     from umls u
#     cross join patients p
# """).df()
# pat_base_df.head()

MemoryError: cannot allocate memory for array

In [114]:
# pat_feat_long_df = con.execute("""
# with patients as (
#     select distinct subject_id from all_pat_df
# )
# ,pat_feat as (
#     select p.subject_id
#     ,u.feature_idx
#     ,u.CUI
#     from umls u
#     cross join patients p
# )
# select pf.subject_id
# ,pf.feature_idx
# ,count(p.umls) feature_val
# from pat_feat pf
# left join all_pat_df p
# on pf.CUI = p.umls
# and pf.subject_id = p.subject_id
# group by pf.subject_id
# ,pf.feature_idx

# /*select p.subject_id
# ,c.feature_idx
# ,count(pc.umls) feature_val
# from umls c
# cross join patients p
# left join all_pat_df pc
# on c.CUI = pc.umls
# and p.subject_id = pc.subject_id
# group by p.subject_id
# ,c.feature_idx*/
# """).df()
# pat_feat_long_df.head()

Error: Invalid Error: bad allocation

In [107]:
# pat_feat_wide = pat_feat_long_df.pivot(index='SUBJECT_ID', columns='feature_idx', values='feature_val')
# pat_feat_wide = pat_feat_wide.reset_index()
# pat_feat_wide.head()

feature_idx,SUBJECT_ID,1,2,3,4,5,6,7,8,9,...,15443,15444,15445,15446,15447,15448,15449,15450,15451,15452
0,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1000,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1001,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# pat_feat_wide.to_csv(os.path.join('data', 'consumed', 'patient_feature.csv'), index=False)

In [26]:
# rel_file = os.path.join('data', 'clinical_vocab', '2022AB', 'META', 'MRREL.RRF')
# cols = ['CUI1', 'AUI1', 'STYPE1', 'REL', 'CUI2', 'AUI2', 'STYPE2', 'RELA', 'RUI',
#     'SRUI', 'SAB', 'SL', 'RG', 'DIR', 'SUPPRESS', 'CVF']
# umls_rel = pd.read_csv(rel_file, sep="|", 
# header=None, names=cols, index_col=False)

# umls_rel.head()

C:\Users\aknof\AppData\Local\Temp\ipykernel_20440\1973826217.py:4: DtypeWarning: Columns (7,9,13) have mixed types. Specify dtype option on import or set low_memory=False.
  umls_rel = pd.read_csv(rel_file, sep="|",


MIMICIII targets

In [6]:
cpt = pd.read_csv(os.path.join('data', 'CPTEVENTS.csv'))
cpt.head()

C:\Users\aknof\AppData\Local\Temp\ipykernel_20440\3658660055.py:1: DtypeWarning: Columns (4,5,7,11) have mixed types. Specify dtype option on import or set low_memory=False.
  cpt = pd.read_csv(os.path.join('data', 'CPTEVENTS.csv'))


,ROW_ID,SUBJECT_ID,HADM_ID,COSTCENTER,CHARTDATE,CPT_CD,CPT_NUMBER,CPT_SUFFIX,TICKET_ID_SEQ,SECTIONHEADER,SUBSECTIONHEADER,DESCRIPTION
0,317,11743,129545,ICU,NaN,99232,99232.0,NaN,6.0,Evaluation and management,Hospital inpatient services,NaN
1,318,11743,129545,ICU,NaN,99232,99232.0,NaN,7.0,Evaluation and management,Hospital inpatient services,NaN
2,319,11743,129545,ICU,NaN,99232,99232.0,NaN,8.0,Evaluation and management,Hospital inpatient services,NaN
3,320,11743,129545,ICU,NaN,99232,99232.0,NaN,9.0,Evaluation and management,Hospital inpatient services,NaN
4,321,6185,183725,ICU,NaN,99223,99223.0,NaN,1.0,Evaluation and management,Hospital inpatient services,NaN


In [7]:
cpt = cpt[['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CPT_CD', 'CPT_NUMBER', 'TICKET_ID_SEQ']]
cpt.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CPT_CD,CPT_NUMBER,TICKET_ID_SEQ
0,317,11743,129545,99232,99232.0,6.0
1,318,11743,129545,99232,99232.0,7.0
2,319,11743,129545,99232,99232.0,8.0
3,320,11743,129545,99232,99232.0,9.0
4,321,6185,183725,99223,99223.0,1.0


In [8]:
dx = pd.read_csv(os.path.join('data', 'DIAGNOSES_ICD.csv'))
dx.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [9]:
proc = pd.read_csv(os.path.join('data', 'PROCEDURES_ICD.csv'))
proc.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,944,62641,154460,3,3404
1,945,2592,130856,1,9671
2,946,2592,130856,2,3893
3,947,55357,119355,1,9672
4,948,55357,119355,2,331


In [10]:
d_cpt = pd.read_csv(os.path.join('data', 'D_CPT.csv'))
d_cpt.head()

,ROW_ID,CATEGORY,SECTIONRANGE,SECTIONHEADER,SUBSECTIONRANGE,SUBSECTIONHEADER,CODESUFFIX,MINCODEINSUBSECTION,MAXCODEINSUBSECTION
0,1,1,99201-99499,Evaluation and management,99201-99216,Office/other outpatient services,NaN,99201,99216
1,2,1,99201-99499,Evaluation and management,99217-99220,Hospital observation services,NaN,99217,99220
2,3,1,99201-99499,Evaluation and management,99221-99239,Hospital inpatient services,NaN,99221,99239
3,4,1,99201-99499,Evaluation and management,99241-99255,Consultations,NaN,99241,99255
4,5,1,99201-99499,Evaluation and management,99261-99263,Follow-up inpatient consultations (deleted codes),NaN,99261,99263


In [11]:
cpt = con.execute("""
select c.SUBJECT_ID
,d.CATEGORY as cpt_cat
from cpt c
join d_cpt d
on c.cpt_number >= d.MINCODEINSUBSECTION
and c.cpt_number <= d.MAXCODEINSUBSECTION 
union
select c.SUBJECT_ID
,c.cpt_cd as cpt_cat
from cpt c
where c.cpt_number is null
""").df()
cpt.head()

,SUBJECT_ID,cpt_cat
0,59231,1
1,43187,1
2,92098,1
3,91383,1
4,69433,1


In [12]:
dx = con.execute("""
select distinct SUBJECT_ID, substring(ICD9_CODE, 1,
    case when left(ICD9_CODE, 1) in ('E', '0') then 4
    else 3 end --V, 1-9
) dx_group
from dx
where ICD9_CODE not like '0%'
and ICD9_CODE not like 'E%'
and ICD9_CODE not like 'V%'
""").df()
dx.head()

,SUBJECT_ID,dx_group
0,24210,424
1,24210,244
2,24211,864
3,24211,682
4,24214,276


In [14]:
pat_tgt_long = con.execute(f"""
with tgts as (
select SUBJECT_ID, cpt_cat as src_val, 'cpt' as src_type
from cpt
union all
select SUBJECT_ID, dx_group, 'dx' as src_type
from dx
union all
select distinct SUBJECT_ID, icd9_code, 'procedure' as src_type
from proc
)
select *, dense_rank() over (order by src_type, src_val) as target_idx
from tgts t
where exists (select 1
                from tgts cnt
                group by src_type, src_val
                having t.src_val = cnt.src_val
                and t.src_type = cnt.src_type
                and count(*) >= {BILL_MIN_OCC_POP})
""").df()
pat_tgt_long.head()

,SUBJECT_ID,src_val,src_type,target_idx
0,59231,1,cpt,1
1,43187,1,cpt,1
2,92098,1,cpt,1
3,91383,1,cpt,1
4,69433,1,cpt,1


In [15]:
target_map = con.execute("""
select src_val, src_type, target_idx 
from pat_tgt_long
group by src_val, src_type, target_idx 
--order by target_idx
""").df()
target_map.head()

,src_val,src_type,target_idx
0,357,dx,234
1,358,dx,235
2,359,dx,236
3,360,dx,237
4,361,dx,238


In [16]:
target_map.to_csv(os.path.join('data', 'consumed', 'target_map.csv'))

In [17]:
target_map.count()

src_val       2803
src_type      2803
target_idx    2803
dtype: int64

In [18]:
pat_tgt_full = con.execute("""
with pat as (select distinct subject_id from pat_tgt_long)
select p.subject_id
,t.target_idx
,case when pt.target_idx is not null then 1 else 0 end as target_val
from target_map t
cross join pat p
left join pat_tgt_long pt
on t.target_idx = pt.target_idx
and p.subject_id = pt.subject_id
""").df()
pat_tgt_full.head()

,SUBJECT_ID,target_idx,target_val
0,60907,282,1
1,60907,284,1
2,60907,321,1
3,60907,324,1
4,60907,353,1


In [19]:
pat_tgt_wide = pat_tgt_full.pivot(index='SUBJECT_ID', columns='target_idx', values='target_val')
pat_tgt_wide = pat_tgt_wide.reset_index()
pat_tgt_wide.head()

target_idx,index,SUBJECT_ID,1,2,3,4,5,6,7,8,...,2794,2795,2796,2797,2798,2799,2800,2801,2802,2803
0,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,3,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,4,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,6,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
pat_tgt_wide = pat_tgt_wide.iloc[:, 1:]
print(pat_tgt_wide.columns[:5], pat_tgt_full.columns)
pat_tgt_wide.head()

Index(['SUBJECT_ID', 1, 2, 3, 4], dtype='object', name='target_idx') Index(['SUBJECT_ID', 'target_idx', 'target_val'], dtype='object')


target_idx,SUBJECT_ID,1,2,3,4,5,6,7,8,9,...,2794,2795,2796,2797,2798,2799,2800,2801,2802,2803
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
pat_tgt_wide.to_csv(os.path.join('data', 'consumed', 'patient_target.csv'), index=False)

In [105]:
# pat_tgt = pd.DataFrame(pat_tgt_wide['SUBJECT_ID'])
# pat_tgt['targets'] = (pat_tgt_wide.iloc[:,2:].astype(str)
#                 .agg(lambda x: ' '.join(i for i in x), axis=1)
#                 .str.split())
# pat_tgt.head()                

,SUBJECT_ID,targets
0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,4,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,6,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [102]:
# pat_tgt.to_csv(os.path.join('data', 'consumed', 'patient_target.csv'), index=False)